In [1]:
import os 
import json
import pandas as pd
import string
import spacy
import re
import numpy as np
from collections import defaultdict

2024-04-03 09:19:49.176806: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-03 09:19:49.179345: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-03 09:19:49.507364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 09:19:52.657928: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


1. convert ether output to input for tf_rel model's input in order
2. apply fine-tuned tf_rel model in order
3. apply rel_label model in order
4. fill the results in dev set
5. get res

In [ ]:
#1. convert ether output to input for tf_rel model's input in order
cancer = "breast"
mode = "dev"

# 1.1 get ether list from timeset/my_code_result/notebooks1/generate_tf_training_input.ipynb
# gold_dct_file = f"/users/the/NER_MTB/0_{cancer}_{mode}_gold_dct.json"
note_dir = os.path.join("/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/input/change_n2space", \
                        f"input_{cancer}_{mode}", "Patient_Notes", f"{cancer}", f"{mode}")
unsummarized_file = f"/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/output/change_n2space/unsummarized_{cancer}_{mode}_output.tsv"
df = pd.read_csv(unsummarized_file, delimiter="\t")

def find_loc(a, b):
    a_clean = ''.join(a.split())
    b_clean = ''.join(b.split())
    index = a_clean.find(b_clean)
    s, out = 0, []
    for i, char in enumerate(a):
        if char == " " or char == "\t":
            continue 
        s+=1 
        if s == index + 1: 
            out.append(i)
            
        if s == index + len(b_clean):
            out.append(i)
    if len(out) == 2:
        end, start = out[0], out[1]+1
        return end, start
    else: 
        return None, None


def _find_surrounding_text(string):
    tlink_inst = string
    e_pattern = r'<e>\s*(.*?)\s*</e>'
    matches = re.findall(e_pattern, tlink_inst)
    return matches

def _find_time_surrounding_text(string):
    tlink_inst = string
    t_pattern = r'<t>\s*(.*?)\s*</t>'
    matches = re.findall(t_pattern, tlink_inst)
    return matches

def match_ignore_spaces(a, b):
    # Create a pattern for string a with optional spaces between each character
    pattern_a = r'\b{}\b'.format(r'\s*'.join(re.escape(word) for word in a.split()))
    # Find the matched pattern in string b
    match = re.search(pattern_a, b)
    if match:
        return match.start(), match.end()
    else:
        return None, None


def _find_time_word_index(original_text, given_string, word):
    original_text_processed = original_text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    word = word.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))

    match_s, match_e = match_ignore_spaces(word, original_text_processed)
    return match_s, match_e

    
def same_characters_in_order(s1, s2):
    # Remove punctuation from both strings
    s1_cleaned = ''.join(char for char in s1 if char not in string.punctuation)
    s2_cleaned = ''.join(char for char in s2 if char not in string.punctuation)
    # Compare the cleaned strings
    return s1_cleaned, s2_cleaned

def decrease_space_before_punctuation(text):
    # Define the regular expression pattern
    pattern = r'(?<=\w)\s+(?=\W)'
    # Replace multiple spaces with a single space before punctuation
    cleaned_text = re.sub(pattern, ' ', text)
    return cleaned_text

def _find_tsv_sentences_in_range(doc, start_index, end_index):
    "Given nth sents of chemo and timex, return the context"
    sentences_in_range = []
    for ind, sent in enumerate(doc.sents):
        if ind >= start_index and ind < end_index:
            sentences_in_range.append(sent)
    return sentences_in_range

def _df_find_nth(lst, match_start_lst, match_end_lst):
    "Used the loc from raw to find where sent it is in Doc"
    nth_sent_lst = []
    ith_start_lst, ith_end_lst = [], []
    if len(lst)!=0:
        for i, chemo in enumerate(lst):
            item_start_ind, item_end_ind = match_start_lst[i], match_end_lst[i]
            if item_end_ind is None or item_start_ind is None:
                print("skip here: ", item_start_ind, item_end_ind )
                item_end_ind, item_start_ind = -1, -1
                # continue

            len_ith = item_end_ind - item_start_ind

            for sent_id, sent in enumerate(doc.sents):
                if sent.start_char <= item_start_ind< sent.end_char: 
                    ith_start_ind = int(item_start_ind - sent.start_char)
                    ith_end_ind = item_start_ind + len_ith
                    for token in sent:
                        if token.idx <= item_start_ind< (token.idx+len(token.text)):
                            nth_sent_lst.append(int(sent_id))
                            ith_start_lst.append(ith_start_ind)
                            ith_end_lst.append(ith_end_ind)
    print("lst", lst)
    print("nth_sent_lst", nth_sent_lst)
    diff = len(lst) - len(nth_sent_lst)
    if len(lst) > len(nth_sent_lst):
        while diff > 0:
            nth_sent_lst.append(-1)
            diff -= 1
    assert len(lst) == len(nth_sent_lst)
    return nth_sent_lst, ith_start_lst, ith_end_lst
    
# with open(gold_dct_file, "r") as infile: 
#     gold_dct = json.load(infile)

spacy_better = spacy.load("en_core_web_sm")

pos_rel = []
final_df = pd.DataFrame()
cnt = 0
# for pat_id in ["patient47"]:
# for pat_id in gold_dct.keys():
#     note_sub_dir = os.path.join(note_dir, pat_id)
#     medium_df = pd.DataFrame()
#     for filename in gold_dct[pat_id].keys():
for pat_id in os.listdir(note_dir):
    if os.path.isdir(os.path.join(note_dir, pat_id)):
        print(pat_id)
        for pat_filename in os.listdir(os.path.join(note_dir, pat_id)):
            filename = "_".join(pat_filename.split(".txt")[0].split("_")[1:])
            print(filename)
            raw_note_file = os.path.join(note_dir, pat_id, pat_filename)
            with open(raw_note_file, "r") as infile: 
                lines = infile.readlines()
            num_pre9_tokens = len(''.join(lines[:9]))
            lines = lines[9:]
            clean_note = "".join(lines)
            doc = spacy_better(clean_note)

            chemo_start_ind_tlinkinst, chemo_end_ind_tlinkinst = [], []
            nth_sent = []
            chemo_lst = []
            time_start_ind_tlinkinst, time_end_ind_tlinkinst = [], []
            time_nth_sent = []
            raw_time_lst = []
            match_sents = []

            selected_i = []
            print(df.loc[df["note_name"]=="patient01_report011_RAD"]["tlink_inst"].iloc[0])
            print(df.loc[df["note_name"]=="patient01_report011_RAD"]["tlink_inst"].iloc[1])
            print(df.loc[df["note_name"]=="patient01_report011_RAD"]["tlink_inst"].iloc[2])
            patient_file = df.loc[df["note_name"] == pat_id + "_" + filename]
            tsv_df = pd.DataFrame()
            selected_row = []
            for i, row in patient_file.iterrows():
                tlink = "none"

                if row["chemo_text"] != "none" and row["normed_timex"] != "none":
                    tlink_inst = row["tlink_inst"]
                    matches = _find_surrounding_text(tlink_inst)
                    time_matches = _find_time_surrounding_text(tlink_inst)
                    if pat_id == "patient01" and filename=="report011_RAD":
                        print(time_matches)

                    selected_row.append(row)
                    
                    if len(matches) == 0 or len(time_matches) == 0:
                        if len(matches) == 0:
                            print("chemo: yese", matches)
                        elif len(time_matches) == 0:
                            print("1time: ")

                    clean_match = matches[0].translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
                    time_clean_match = time_matches[0].translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
                    
                    chemo_lst.append(clean_match)
                    raw_time_lst.append(time_clean_match) # not exact raw time, but if use the index it is fine
                    
                    start_ind, end_ind = find_loc(clean_note, clean_match)
                    time_start_ind, time_end_ind = _find_time_word_index(clean_note, tlink_inst, time_clean_match)

                    chemo_start_ind_tlinkinst.append(start_ind)
                    chemo_end_ind_tlinkinst.append(end_ind)
                    time_start_ind_tlinkinst.append(time_start_ind)
                    time_end_ind_tlinkinst.append(time_end_ind)
    
            nth_sent_item, ith_start_item, ith_end_item = _df_find_nth(chemo_lst, chemo_start_ind_tlinkinst, chemo_end_ind_tlinkinst) 
            time_nth_sent_item, time_ith_start_item, time_ith_end_item = _df_find_nth(raw_time_lst, time_start_ind_tlinkinst, time_end_ind_tlinkinst)

            tsv_df["chemo_tlinkinst_start"] = chemo_start_ind_tlinkinst
            tsv_df["chemo_tlinkinst_end"] = chemo_end_ind_tlinkinst
            tsv_df["nth_sent"] = nth_sent_item

            tsv_df["time_tlinkinst_start"] = time_start_ind_tlinkinst
            tsv_df["time_tlinkinst_end"] = time_end_ind_tlinkinst
            tsv_df["time_nth_sent"] = time_nth_sent_item
            tsv_df["time_ith_start"] = time_ith_start_item
            tsv_df["time_ith_end"] = time_ith_end_item
            tsv_df["raw_time"] = raw_time_lst

            sent_range_lst = []
            for i in range(len(time_nth_sent_item)):
                if int(nth_sent_item[i]) > int(time_nth_sent_item[i]):
                    start_nth_sent = int(time_nth_sent_item[i])
                    end_nth_sent = int(nth_sent_item[i])
                else: 
                    start_nth_sent = int(nth_sent_item[i])
                    end_nth_sent = int(time_nth_sent_item[i])
                sent_range = _find_tsv_sentences_in_range(doc, start_nth_sent, end_nth_sent+1)
                sent_range_lst.append(sent_range) 
            tsv_df["match_sents"] = sent_range_lst
            selected_pat_file = patient_file.iloc[selected_i]
            selected_pat_file = pd.DataFrame(columns=["DCT", "patient_id", "chemo_text", "chemo_annotation_id", "normed_timex", "timex_annotation_id", "tlink", "note_name", "tlink_inst"], data=selected_row)
            new_df = pd.concat([selected_pat_file.reset_index(), tsv_df.reset_index()], axis=1)
            medium_df = pd.concat([new_df, medium_df])
    final_df = pd.concat([final_df, medium_df])
final_df = final_df.reset_index()

# final_df.to_csv(f"./inference/{cancer}_{mode}_wo_labels.tsv", sep="\t", index=False)



In [ ]:
print(final_df[final_df["note_name"]=="patient01_report011_RAD"])

In [ ]:
note_dir = os.path.join("/users/the/NER_MTB/timelines/chemoTimelinesBaselineSystem/input/change_n2space", \
                        f"input_{cancer}_{mode}", "Patient_Notes", f"{cancer}", f"{mode}")
tsv_lst = []
pat_filename = np.unique(final_df["note_name"]).tolist()
for uni_pat_filename in pat_filename:
    single_dct = defaultdict(dict)
    single_df = final_df[final_df["note_name"] == uni_pat_filename]
    # print(single_df.columns)
    print(single_df)
    for i, row in single_df.iterrows():
        print(i, row)
        if abs(row["nth_sent"]-row["time_nth_sent"]) <= 1:
            single_dct["filename"] = uni_pat_filename
            single_dct["statement"] = "Does the arg1 and arg2 has relationship?"
            single_dct["arg1"] = {
                "mention": row["chemo_text"],
                "nth_sent": int(row["nth_sent"])
            }
            single_dct["arg2"] = {
                "mention": row["raw_time"],
                "time_tlinkinst_start": int(row["time_tlinkinst_start"]),
                "time_tlinkinst_end": int(row["time_tlinkinst_end"]),
                "time_nth_sent": int(row["time_nth_sent"])
            }
            
            pat_id = uni_pat_filename.split("_")[0]
            note_sub_dir = os.path.join(note_dir, pat_id)
            pat_filename_txt = uni_pat_filename+".txt"
            raw_note_file = os.path.join(note_sub_dir, pat_filename_txt)

            with open(raw_note_file, "r") as infile: 
                lines = infile.readlines()
            num_pre9_tokens = len(''.join(lines[:9]))
            single_dct["num_pre9_tokens"] = num_pre9_tokens 

            lines = lines[9:]
            clean_note = "".join(lines)
            doc = spacy_better(clean_note)
            
            if int(row["nth_sent"]) < int(row["time_nth_sent"]):
                start_sent_ind = int(row["nth_sent"])
                end_start_ind = int(row["time_nth_sent"])
            else: 
                start_sent_ind = int(row["time_nth_sent"])
                end_start_ind = int(row["nth_sent"])
            
            temp = []
            for i, sent in enumerate(doc.sents):
                if i<= end_start_ind and i >= start_sent_ind:
                    temp.append(sent.text)
            single_dct["context"] = " ".join(temp)
            single_dct["tlink"] = "BEGINS-ON" #"none"
            single_dct["rel_type"] = "positive"
            print(single_dct)
            tsv_lst.append(single_dct)

def _decrease_space_before_punctuation(text):
    # Define the regular expression pattern
    pattern = r'(?<=\w)\s+(?=\W)'
    # Replace multiple spaces with a single space before punctuation
    cleaned_text = re.sub(pattern, ' ', text)
    return cleaned_text


# def _further_clean_json(dir, cancer, mode):
def _further_clean_json(tsv_lst):
    
    final_neg_lst = []
    
    #infile = f"/users/the/NER_MTB/timeset/my_code_result/input_data/tf_rel_task/inference/{cancer}/{mode}/test.json"
    # infile = os.path.join(dir, cancer, mode, "test.json")
    # with open(infile, "r") as file: 
    #     input_lst = json.load(file)

    for neg in tsv_lst:
        context = neg["context"]
        ment1 = neg["arg1"]["mention"]
        ment2 =neg["arg2"]["mention"]
        cleaned_context = _decrease_space_before_punctuation(context)
        cleaned_ment1, cleaned_ment2 = _decrease_space_before_punctuation(ment1), _decrease_space_before_punctuation(ment2)
        final_neg_lst.append({
            "filename" :neg["filename"],
            "statement" :neg["statement"],
            "label":neg["label"],
            "relation":neg["relation"],
            # "num_pre9_tokens":neg["num_pre9_tokens"],
            # "rel_type":"BEGINS-ON",
            # "label":"positive",
            "arg1":{
                "mention":cleaned_ment1,
                # "nth_sent":neg["arg1"]["nth_sent"],
                "start":find_loc(cleaned_context,cleaned_ment1)[0],
                "end":find_loc(cleaned_context,cleaned_ment1)[1]
            },
            "arg2":{
                "mention":cleaned_ment2,
                # "nth_sent":neg["arg2"]["nth_sent"],
                "start":find_loc(cleaned_context,cleaned_ment2)[0],
                "end":find_loc(cleaned_context,cleaned_ment2)[1]
            }, "context":cleaned_context

        })

final_lst = _further_clean_json(tsv_lst)

In [ ]:
tsv_lst

In [ ]:
def find_loc(a, b):
    # a: long one
    a = re.sub('[^0-9a-zA-Z]', ' ', a)
    b = re.sub('[^0-9a-zA-Z]', ' ', b)
    a_clean = ''.join(a.split())
    b_clean = ''.join(b.split())
    index = a_clean.find(b_clean)
    s, out = 0, []
    for i, char in enumerate(a):
        if char == " " or char == "\t":
            continue 
        s+=1 
        if s == index + 1: 
            out.append(i)           
        if s == index + len(b_clean):
            out.append(i)
    if len(out) == 2:
        start, end = out[0], out[1]+1
        return start, end
    else: 
        return None, None

final_neg_lst = []

for neg in tsv_lst:
    context = neg["context"]

    ment1 = neg["arg1"]["mention"]
    ment2 =neg["arg2"]["mention"]
    final_neg_lst.append({
        "filename" :neg["filename"],
        "statement" :neg["statement"],
        "num_pre9_tokens":neg["num_pre9_tokens"],
        "rel_type":"BEGINS-ON",
        "label":"positive",
        "arg1":{
            "mention":neg["arg1"]["mention"],
            "nth_sent":neg["arg1"]["nth_sent"],
            "start":find_loc(context,ment1)[0],
            "end":find_loc(context,ment1)[1]
        },
        "arg2":{
            "mention":neg["arg2"]["mention"],
            "nth_sent":neg["arg2"]["time_nth_sent"],
            "start":find_loc(context,ment2)[0],
            "end":find_loc(context,ment2)[1]
        }, "context":context

    })

    final_neg_lst

# with open(f"/users/the/NER_MTB/timeset/my_code_result/model/tf_rel_task/temporal-nli/inference/{cancer}_{mode}_train_tf_input.json", "w") as outfile: 
with open(f"/users/the/NER_MTB/timeset/my_code_result/input_data/tf_rel_task/inference/{cancer}/{mode}/test.json", "w") as outfile: 
    json.dump(final_neg_lst, outfile, indent=4)
# test.json

In [ ]:
final_neg_lst

In [68]:
# 2. apply fine-tuned tf_rel model in order

# /users/the/NER_MTB/timeset/my_code_result/scripts/benchmark/tf_rel_task/inference_ft_deberta-v3-large_bfloat16.sh

In [78]:
# need to change the code later

tf_rel_lst = []

for rel in final_neg_lst:
    context = rel["context"]
    ment1 = rel["arg1"]["mention"]
    ment2 =rel["arg2"]["mention"]
    tf_rel_lst.append({
        "filename" :rel["filename"],
        "statement" :rel["statement"],
        "num_pre9_tokens":rel["num_pre9_tokens"],
        "rel_type":"BEGINS-ON",
        "label":"positive",
        "arg1":{
            "mention":rel["arg1"]["mention"],
            "nth_sent":rel["arg1"]["nth_sent"],
            "start":rel["arg1"]["start"],
            "end":rel["arg1"]["end"]
        },
        "arg2":{
            "mention":rel["arg2"]["mention"],
            "nth_sent":rel["arg2"]["nth_sent"],
            "start":rel["arg2"]["start"],
            "end":rel["arg2"]["end"]
        }, "context":rel["context"]

    })

    tf_rel_lst


temp_file = f"/users/the/NER_MTB/timeset/my_code_result/input_data/tf_rel_task/inference/{cancer}/{mode}"
os.makedirs(temp_file, exist_ok=True)

with open(os.path.join(f"{temp_file}/test.json"), "w") as outfile: 
    json.dump(tf_rel_lst, outfile, indent=4)

In [19]:
# further clean dataset by removing the space 
cancer = "breast"
mode = "dev"

def find_loc(a, b):
    # a: long one
    a = re.sub('[^0-9a-zA-Z]', ' ', a)
    b = re.sub('[^0-9a-zA-Z]', ' ', b)
    a_clean = ''.join(a.split())
    b_clean = ''.join(b.split())
    index = a_clean.find(b_clean)
    s, out = 0, []
    for i, char in enumerate(a):
        if char == " " or char == "\t":
            continue 
        s+=1 
        if s == index + 1: 
            out.append(i)           
        if s == index + len(b_clean):
            out.append(i)
    if len(out) == 2:
        start, end = out[0], out[1]+1
        return start, end
    else: 
        return None, None

def _decrease_space_before_punctuation(text):
    # Define the regular expression pattern
    pattern = r'(?<=\w)\s+(?=\W)'
    # Replace multiple spaces with a single space before punctuation
    cleaned_text = re.sub(pattern, ' ', text)
    return cleaned_text


def _further_clean_json(dir, cancer, mode):
    
    final_neg_lst = []
    
    #infile = f"/users/the/NER_MTB/timeset/my_code_result/input_data/tf_rel_task/inference/{cancer}/{mode}/test.json"
    infile = os.path.join(dir, cancer, mode, "test.json")
    with open(infile, "r") as file: 
        input_lst = json.load(file)

    for neg in input_lst:
        context = neg["context"]
        ment1 = neg["arg1"]["mention"]
        ment2 =neg["arg2"]["mention"]
        cleaned_context = _decrease_space_before_punctuation(context)
        cleaned_ment1, cleaned_ment2 = _decrease_space_before_punctuation(ment1), _decrease_space_before_punctuation(ment2)
        final_neg_lst.append({
            "filename" :neg["filename"],
            "statement" :neg["statement"],
            "label":neg["label"],
            "relation":neg["relation"],
            # "num_pre9_tokens":neg["num_pre9_tokens"],
            # "rel_type":"BEGINS-ON",
            # "label":"positive",
            "arg1":{
                "mention":cleaned_ment1,
                # "nth_sent":neg["arg1"]["nth_sent"],
                "start":find_loc(cleaned_context,cleaned_ment1)[0],
                "end":find_loc(cleaned_context,cleaned_ment1)[1]
            },
            "arg2":{
                "mention":cleaned_ment2,
                # "nth_sent":neg["arg2"]["nth_sent"],
                "start":find_loc(cleaned_context,cleaned_ment2)[0],
                "end":find_loc(cleaned_context,cleaned_ment2)[1]
            }, "context":cleaned_context

        })

        final_neg_lst
    # with open(f"/users/the/NER_MTB/timeset/my_code_result/input_data/tf_rel_task/inference/{cancer}/{mode}/test.json", "w") as outfile:
    with open(infile, "w") as outfile: 
        json.dump(final_neg_lst, outfile, indent=4)

dir = "/users/the/NER_MTB/timeset/my_code_result/input_data/tf_rel_task/inference"
_further_clean_json(dir, cancer, mode)

In [3]:
# 3. apply rel_label model in order
# need to change the code later
cancer = "breast"
mode = "dev"

with open(f"/users/the/NER_MTB/timeset/my_code_result/input_data/tf_rel_task/inference/{cancer}/{mode}/test.json", "r") as infile: 
    final_neg_lst = json.load(infile) 

tf_rel_lst = []

for rel in final_neg_lst:
    context = rel["context"]
    ment1 = rel["arg1"]["mention"]
    ment2 =rel["arg2"]["mention"]
    tf_rel_lst.append({
        "filename" :rel["filename"],
        # "statement" :rel["statement"],
        "relation":"BEGINS-ON",
        # "label":"positive",
        "arg1":{
            "mention":rel["arg1"]["mention"],
            "start":rel["arg1"]["start"],
            "end":rel["arg1"]["end"]
        },
        "arg2":{
            "mention":rel["arg2"]["mention"],
            "start":rel["arg2"]["start"],
            "end":rel["arg2"]["end"]
        }, "context":rel["context"]

    })

    tf_rel_lst


temp_file = f"/users/the/NER_MTB/timeset/my_code_result/input_data/rel_label_task/inference/{cancer}/{mode}"
os.makedirs(temp_file, exist_ok=True)

with open(os.path.join(f"{temp_file}/test.json"), "w") as outfile: 
    json.dump(tf_rel_lst, outfile, indent=4)

